# 1. 
    1.1 模型处理
        1.2 获取数据
        1.3 模型
        1.4 多个模型
        1.5 融合
        

In [2]:
#import jieba 
import gensim
import pandas as pd
import numpy as np
import lightgbm
import sklearn
import torch
from torch import nn
from sklearn.metrics import f1_score
import torch.nn.functional as F

In [3]:
doc2vec=gensim.models.Doc2Vec.load('../model/model_doc')

In [ ]:
doc2vec.docvecs

In [4]:
def read2():
    '''
    第一次读后的结果保存之后在读取
    '''
    file=r'../data/nlp_train.csv'
    train=pd.read_csv(file,sep='\t',encoding='utf-8',header=None,index_col=None)
    return train
train=read2()

In [5]:
def read_sentences():
    '''
    读取分割好的句子
    '''
    file='../data/sentences.txt'
    sentences=pd.read_table(file,encoding='utf-8',header=None)
    return sentences
sentences=read_sentences()

In [6]:
train.head()

,0,1,2,3
0,1,﻿怎么更改花呗手机号码,我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号,1
1,2,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0
2,3,花呗冻结以后还能开通吗,我的条件可以开通花呗借款吗,0
3,4,如何得知关闭借呗,想永久关闭借呗,0
4,5,花呗扫码付钱,二维码扫描可以用花呗吗,0


In [7]:
sentences.head()

,0
0,怎么 更改 花呗 手机号码
1,也 开 不了 花呗 ， 就 这样 了 ？ 完事 了
2,花呗 冻结 以后 还 能 开通 吗
3,如何 得知 关闭 借呗
4,花呗 扫码 付钱


In [8]:
def cal_train_x(sentences,model):
    '''
    再把分词后的句子转换成矢量
    '''
    train_x=[]
    split=len(sentences)//2
    train_x_q1=[model.infer_vector(i) for i in sentences[:split].values]
    train_x_q2=[model.infer_vector(i) for i in sentences[-split:].values]
    train_x=[np.concatenate((train_x_q1[i],train_x_q2[i])) for i in range(len(train_x_q1))]
    return np.array(train_x)
train_x=cal_train_x(sentences,doc2vec)

In [9]:
train_x.shape,train.shape

((102477, 200), (102477, 4))

### 2. lightgbm

In [10]:
train_y=train.loc[:,3]
train_y.shape

(102477,)

In [112]:
f1_score?

In [21]:
import time

def LGB_predict(train_x,train_y,):
    
    print("LGB test")
    clf = lightgbm.LGBMClassifier(
        boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
        max_depth=-1, n_estimators=20000, objective='binary',
        subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
        learning_rate=0.05, min_child_weight=50, random_state=2008, n_jobs=-1
    )
    n_split=10
    cv_split = sklearn.model_selection.KFold(n_splits=n_split, random_state=15, shuffle=False)

    #保存每个交叉验证在y_test集上的结果
    #y_pred_test_cv=np.ones((test_x.shape[0],n_split))#
    #y_pred_train_cv=np.ones((train_x.shape[0],))    
    for i, (train_index,test_index) in enumerate(cv_split.split(train_x,train_y)):     
        clf.fit(train_x[train_index],train_y[train_index],eval_set=[(train_x[test_index],train_y[test_index])], \
                eval_metric='binary', verbose=2000)
        y_pred=clf.predict(train_x[test_index])         
        #print(y_pred)
        #print(train_y[test_index].values)
        print(i,f1_score(train_y[test_index],y_pred))
    return clf
clf=LGB_predict(train_x,np.array(train_y))

LGB test
[2000]	valid_0's binary_logloss: 0.536116
[4000]	valid_0's binary_logloss: 0.543005
[6000]	valid_0's binary_logloss: 0.551537
[8000]	valid_0's binary_logloss: 0.560027
[10000]	valid_0's binary_logloss: 0.570634
[12000]	valid_0's binary_logloss: 0.58213
[14000]	valid_0's binary_logloss: 0.594301
[16000]	valid_0's binary_logloss: 0.607262
[18000]	valid_0's binary_logloss: 0.620563
[20000]	valid_0's binary_logloss: 0.63405
0 0.00897263346792
[2000]	valid_0's binary_logloss: 0.539035
[4000]	valid_0's binary_logloss: 0.547512
[6000]	valid_0's binary_logloss: 0.555521
[8000]	valid_0's binary_logloss: 0.566049
[10000]	valid_0's binary_logloss: 0.576777
[12000]	valid_0's binary_logloss: 0.589007
[14000]	valid_0's binary_logloss: 0.602032
[16000]	valid_0's binary_logloss: 0.614998
[18000]	valid_0's binary_logloss: 0.629537
[20000]	valid_0's binary_logloss: 0.643766
1 0.00266785237884
[2000]	valid_0's binary_logloss: 0.536519
[4000]	valid_0's binary_logloss: 0.544965
[6000]	valid_0's bi

In [28]:
doc2vec.most_similar('欠')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('借', 0.4608169496059418),
 ('我差', 0.44239649176597595),
 ('永', 0.43275049328804016),
 ('享用', 0.4084711968898773),
 ('剩', 0.39401108026504517),
 ('差', 0.38787755370140076),
 ('借点', 0.38490962982177734),
 ('借出', 0.3805862367153168),
 ('要换', 0.3681598901748657),
 ('刚借', 0.36452716588974)]